In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import orjson as json
from transformers import AutoTokenizer

In [3]:
with open('test.jsonl', 'w') as fopen_l:
    with open('fb-dedup.jsonl') as fopen:
        for l in fopen:
            l = json.loads(l)
            data = {
                'text': l
            }
            fopen_l.write(f'{json.dumps(data).decode()}\n')

In [4]:
tokenizer = AutoTokenizer.from_pretrained('./out-super-tiny-1.1')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from utils.copied_utils import (
    compute_input_and_target_lengths,
    DataCollatorForT5MLM,
    tokenize_function,
    DataCollatorForNI,
)

/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [6]:
filename = 'test.jsonl'

In [7]:
import datasets

dataset = datasets.load_dataset('json', data_files=filename, split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/husein/.cache/huggingface/datasets/json/default-ea880e3901ec65ab/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


In [8]:
dataset

Dataset({
    features: ['text'],
    num_rows: 193375
})

In [9]:
before_mask_input_length, target_length = compute_input_and_target_lengths(
    inputs_length=512,
    noise_density=0.15,
    mean_noise_span_length=3.0,
)
before_mask_input_length, target_length

(568, 114)

In [10]:
dataset = dataset.map(
    tokenize_function,
    batched=True,
    fn_kwargs={
        'tokenizer': tokenizer,
        'in_length': before_mask_input_length,
    },
    remove_columns=['text'],
    load_from_cache_file=True,
    cache_file_name=f'./{filename}-tokenized',
)

Map:   0%|          | 0/193375 [00:00<?, ? examples/s]

In [18]:
len(dataset[0]['input_ids'])

568